In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString                  as B
import qualified Data.ByteString.Char8            as B (lines)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

input <- B.lines <$> B.readFile "../input/day25.txt"

In [3]:
import Data.List

data Register = A | B | C | D deriving (Eq, Show, Ord)

data Instruction
    = Cpy (Either Int Register) Register
    | Inc Register
    | Dec Register
    | Jnz (Either Int Register) (Either Int Register)
    | Out Register
    | Invalid Instruction
    deriving (Eq, Show)

parseRegister = A.choice
    [ A.string "a" *> (return A)
    , A.string "b" *> (return B)
    , A.string "c" *> (return C)
    , A.string "d" *> (return D)
    ]

parseCpy = do
    A.string "cpy"
    AC.space
    value <- A.choice [Left <$> AC.signed AC.decimal, Right <$> parseRegister]
    AC.space
    reg <- parseRegister
    return $ Cpy value reg

parseInc = do
    A.string "inc"
    AC.space
    reg <- parseRegister
    return $ Inc reg

parseDec = do
    A.string "dec"
    AC.space
    reg <- parseRegister
    return $ Dec reg

parseJnz = do
    A.string "jnz"
    AC.space
    reg <- A.choice [Left <$> AC.signed AC.decimal, Right <$> parseRegister]
    AC.space
    jmp <- A.choice [Left <$> AC.signed AC.decimal, Right <$> parseRegister]
    return $ Jnz reg jmp

parseOut = do
    A.string "out"
    AC.space
    reg <- parseRegister
    return $ Out reg

parseInstruction = A.choice [parseCpy, parseInc, parseDec, parseJnz, parseOut]

parsed = map (either error id . A.parseOnly parseInstruction) input
mapM_ print parsed

Cpy (Right A) D
Cpy (Left 15) C
Cpy (Left 170) B
Inc D
Dec B
Jnz (Right B) (Left (-2))
Dec C
Jnz (Right C) (Left (-5))
Cpy (Right D) A
Jnz (Left 0) (Left 0)
Cpy (Right A) B
Cpy (Left 0) A
Cpy (Left 2) C
Jnz (Right B) (Left 2)
Jnz (Left 1) (Left 6)
Dec B
Dec C
Jnz (Right C) (Left (-4))
Inc A
Jnz (Left 1) (Left (-7))
Cpy (Left 2) B
Jnz (Right C) (Left 2)
Jnz (Left 1) (Left 4)
Dec B
Dec C
Jnz (Left 1) (Left (-4))
Jnz (Left 0) (Left 0)
Out B
Jnz (Right A) (Left (-19))
Jnz (Left 1) (Left (-21))

In [62]:
-- import Control.Monad.Trans.State
import qualified Data.Map.Strict as M
import qualified Data.Vector     as V

type Registers = M.Map Register Int
data Computer = Computer
    { pointer      :: Int
    , registers    :: Registers
    } deriving (Eq, Show)

-- {-- Peephole optimisations
runInstr (Computer 3 rs) _ = let
    d'   = (rs M.! D) + (rs M.! B)
    rs'  = M.insert D d' rs
    rs'' = M.insert B 0  rs'
    in Computer 6 rs''

runInstr (Computer 1 rs) _ = let
    d' = (15*170) + (rs M.! D)
    rs' = M.insert D d' rs
    in Computer 8 rs'


-- runInstr (Computer 0 rs) _ = let
--    d' = (rs M.! A) + (15*170)
    
    

-- -}

runInstr (Computer p rs) (Cpy (Left value) reg) = let
    rs' = M.insert reg value rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Cpy (Right reg1) reg2) = let
    value = rs M.! reg1
    rs'   = M.insert reg2 value rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Inc reg) = let
    value = rs M.! reg
    rs'   = M.insert reg (value+1) rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Dec reg) = let
    value = rs M.! reg
    rs'   = M.insert reg (value-1) rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Jnz (Left value) jmp)
    | value == 0       = Computer (p+1) rs
    | (Left v)  <- jmp = Computer (p+v) rs
    | (Right r) <- jmp = let
        v = rs M.! r
        in               Computer (p+v) rs
runInstr (Computer p rs) (Jnz (Right reg) jmp)
    | (rs M.! reg) == 0 = Computer (p+1) rs
    | (Left v)  <- jmp = Computer (p+v) rs
    | (Right r) <- jmp = let
        v = rs M.! r
        in               Computer (p+v) rs

runOut c@(Computer p rs) (Out reg) o = let
    value = rs M.! reg
    c' = Computer (p+1) rs
    o' = o ++ [value]
    in (c', o')

run c                  instructions count output | length output == 10 = (c,output)
run c@(Computer p rs)  instructions count output 
    | p >= V.length instructions = (c,output)
    | otherwise = let
        i  = instructions V.! p
        in case i of
            Out r -> let
                (c', output') = runOut c i output
                in run c' instructions count output'
            _ -> let
                c' = runInstr c i
                in run c' instructions count output

start = Computer 0 $ M.fromList [(A, 2), (B, 0), (C, 0), (D, 0)]
run start (V.fromList parsed) 0 []

(Computer {pointer = 28, registers = fromList [(A,2),(B,0),(C,0),(D,2552)]},[0,0,0,1,1,1,1,1,1,0])

In [63]:
makeStart n = Computer 0 $ M.fromList [(A, n), (B, 0), (C, 0), (D, 0)]

signal = 0 : 1 : signal

ten = take 10 signal
ten


import Data.List

find (\i -> let
    state = makeStart i
    (c,o) = run state (V.fromList parsed) 0 []
    in o == ten) [1..1000]

[0,1,0,1,0,1,0,1,0,1]

Just 180

6699